In [138]:
import pandas as pd

In [139]:
CatData = pd.read_csv("FinalCatData.csv",low_memory=False)

In [140]:
#setting up a string of the most commonly used words in the english language. 
commonENwords = ['the','be','to','of','and','in','that','have','no','none','is','No','None','No suggestions']
commonENwords = '|'.join(commonENwords)

In [141]:
commonENwords

'the|be|to|of|and|in|that|have|no|none|is|No|None|No suggestions'

In [142]:
CatData['HasEnglish'] = CatData.Customer_Recommendations.str.contains(commonENwords)

In [150]:
#subsetting the data down to all languages sans english and for those rows that don't contain english words listed above
CatDataNonEnglish = CatData[(CatData['HasEnglish']==False) & (CatData['LANGUAGE_ISO_CODE']!='en')]

In [109]:
#subsetting the data further to create a data frame that only contains the column we want to translate
CatDataNonEnglishForAPI = pd.DataFrame(CatDataNonEnglish['Customer_Recommendations'])

In [113]:
#getting the row count needed for the for loop 
RowCount = len(CatDataNonEnglishForAPI.index)

In [115]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

In [116]:
#creating an empty list to be filled by API. 
translation = [None]*RowCount

In [118]:
#running the API below through a for loop to pass each cell individually
for i in range(RowCount):
    translation[i] = translate_client.translate(CatDataNonEnglishForAPI.iloc[i,0],target_language='en')

In [119]:
#putting the data back into a dataframe
translationDF = pd.DataFrame(translation)

In [129]:
#merging the translations back with the dataframe that was subsetted to non english. Know that each row will line up, thus 
#concat in this case will work properly. 
CatDataNonEnglishForAPIwithtranslations = pd.concat([CatDataNonEnglish.reset_index(drop=True), translationDF], axis=1)

In [132]:
#narrowing the dataframe above to columns that we'll use to to a left join with the orignal dataset
ForMerging = CatDataNonEnglishForAPIwithtranslations[['_recordId', 'translatedText']].copy()

In [135]:
#doing a left join to merge our translated text with the original dataframe
CatData = pd.merge(CatData, ForMerging, how='left', on='_recordId')